# 🎮 Fine-tuning Mistral 7B pour GW2 WvW Counter-Picker

Ce notebook permet de fine-tuner le modèle Mistral 7B sur les données de combats GW2 WvW.

**Prérequis** :
- Google Colab (gratuit)
- GPU T4 (15GB VRAM - suffisant avec quantification 4-bit)
- ~45-60 minutes pour le fine-tuning

**Fonctionnalités** :
- ✅ Checkpoints automatiques (reprise après interruption)
- ✅ Sauvegarde sur Google Drive
- ✅ Export GGUF pour Ollama
- ✅ Specs Visions of Eternity incluses

## 1️⃣ Configuration et dépendances

In [ ]:
# Vérifier le GPU disponible
!nvidia-smi

import torch
print(f"\n✓ PyTorch version: {torch.__version__}")
print(f"✓ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✓ VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Monter Google Drive pour sauvegarder les checkpoints
from google.colab import drive
drive.mount('/content/drive')

# Créer le dossier de sauvegarde
import os
SAVE_DIR = "/content/drive/MyDrive/GW2_FineTuning"
CHECKPOINT_DIR = f"{SAVE_DIR}/checkpoints_mistral"
GGUF_DIR = f"{SAVE_DIR}/mistral-7b-gw2-gguf"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(GGUF_DIR, exist_ok=True)
print(f"✓ Dossier de sauvegarde: {SAVE_DIR}")

In [ ]:
# Installer Unsloth
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes triton
!pip install datasets huggingface_hub

print("✓ Dépendances installées")

## 2️⃣ Charger le dataset GW2 WvW

In [ ]:
# Uploader le dataset depuis ton PC
from google.colab import files
import shutil

# Vérifier si le dataset existe déjà sur Drive
DATASET_PATH = f"{SAVE_DIR}/finetune_dataset_mistral.jsonl"

if os.path.exists(DATASET_PATH):
    print(f"✓ Dataset trouvé sur Drive: {DATASET_PATH}")
    shutil.copy(DATASET_PATH, "finetune_dataset_mistral.jsonl")
else:
    print("📁 Upload le fichier 'finetune_dataset_mistral.jsonl' depuis ton PC:")
    uploaded = files.upload()
    # Sauvegarder sur Drive pour les prochaines fois
    for filename in uploaded.keys():
        shutil.copy(filename, DATASET_PATH)
        print(f"✓ Dataset sauvegardé sur Drive")

In [ ]:
# Charger et préparer le dataset
from datasets import load_dataset

dataset = load_dataset("json", data_files="finetune_dataset_mistral.jsonl", split="train")

print(f"✓ Dataset chargé: {len(dataset)} exemples")
print(f"\n📋 Exemple:")
print(f"Instruction: {dataset[0]['instruction'][:200]}...")
print(f"Output: {dataset[0]['output']}")

In [ ]:
# Formater le dataset pour Mistral (format [INST])
def format_prompt(example):
    instruction = example['instruction']
    if not instruction.startswith('[INST]'):
        instruction = f"[INST] {instruction} [/INST]"
    return {
        "text": f"{instruction}\n{example['output']}</s>"
    }

formatted_dataset = dataset.map(format_prompt)
print(f"✓ Dataset formaté pour Mistral")

## 3️⃣ Charger le modèle Mistral 7B

In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 2048
dtype = None
load_in_4bit = True  # IMPORTANT: 4-bit pour tenir dans 15GB VRAM

# Vérifier si un checkpoint existe
checkpoint_exists = os.path.exists(f"{CHECKPOINT_DIR}/checkpoint-latest")

if checkpoint_exists:
    print("🔄 Checkpoint trouvé! Reprise de l'entraînement...")
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=f"{CHECKPOINT_DIR}/checkpoint-latest",
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit,
    )
else:
    print("📥 Chargement du modèle Mistral-7B-Instruct-v0.3...")
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="mistralai/Mistral-7B-Instruct-v0.3",
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit,
    )

print(f"✓ Modèle chargé")

In [ ]:
# Ajouter les adaptateurs LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

print(f"✓ Adaptateurs LoRA ajoutés")

## 4️⃣ Fine-tuning avec checkpoints automatiques

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Configuration avec checkpoints - batch size réduit pour Mistral 7B
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=1,  # Réduit pour Mistral 7B
        gradient_accumulation_steps=8,  # Compensé par plus d'accumulation
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        output_dir=CHECKPOINT_DIR,
        # CHECKPOINTS - Sauvegarde toutes les 50 steps
        save_strategy="steps",
        save_steps=50,
        save_total_limit=3,
        resume_from_checkpoint=True if checkpoint_exists else None,
        report_to="none",
    ),
)

print(f"✓ Trainer configuré avec checkpoints automatiques")
print(f"✓ Sauvegarde toutes les 50 steps dans: {CHECKPOINT_DIR}")

In [ ]:
# 🚀 Lancer le fine-tuning
print("🚀 Démarrage du fine-tuning Mistral 7B...")
print("⏱️ Durée estimée: 45-60 minutes sur GPU T4")
print("💾 Checkpoints sauvegardés sur Google Drive (reprise automatique si interruption)")
print("-" * 50)

trainer_stats = trainer.train(resume_from_checkpoint=checkpoint_exists)

print("-" * 50)
print(f"✓ Fine-tuning terminé!")
print(f"✓ Loss finale: {trainer_stats.training_loss:.4f}")

In [ ]:
# Sauvegarder le checkpoint final
model.save_pretrained(f"{CHECKPOINT_DIR}/checkpoint-latest")
tokenizer.save_pretrained(f"{CHECKPOINT_DIR}/checkpoint-latest")
print(f"✓ Checkpoint final sauvegardé sur Drive")

## 5️⃣ Tester le modèle fine-tuné

In [ ]:
FastLanguageModel.for_inference(model)

test_prompt = """[INST] Guild Wars 2 WvW counter-picker.

VALID SPECS: Dragonhunter, Firebrand, Willbender, Luminary, Berserker, Spellbreaker, Bladesworn, Paragon, Scrapper, Holosmith, Mechanist, Amalgam, Druid, Soulbeast, Untamed, Galeshot, Daredevil, Deadeye, Specter, Antiquary, Tempest, Weaver, Catalyst, Evoker, Chronomancer, Mirage, Virtuoso, Troubadour, Reaper, Scourge, Harbinger, Ritualist, Herald, Renegade, Vindicator, Conduit

Mode: ZERG (25+ players)
Enemy: 4x Firebrand, 3x Scourge, 2x Scrapper, 2x Spellbreaker, 1x Paragon

[ENEMY ANALYSIS]
- Firebrand: support, heal, stability (weak to: boon strip, boon corrupt)
- Scourge: condi, corrupt, barrier (weak to: burst, focus fire)
- Scrapper: support, superspeed, cleanse (weak to: boon strip, focus fire)
- Spellbreaker: frontline, strip, cc (weak to: condi pressure, kiting)
- Paragon: support, chants, stability (weak to: boon corrupt, burst)

Respond EXACTLY in this format:
CONTER: Nx Spec, Nx Spec
FOCUS: Target1 > Target2
TACTIQUE: One tactical advice [/INST]"""

inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    temperature=0.1,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id,
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("📋 Test du modèle fine-tuné:")
print("=" * 50)
print(response.split("[/INST]")[-1].strip())

## 6️⃣ Exporter en GGUF pour Ollama

**Note**: L'export se fait directement sur Google Drive pour éviter les erreurs d'espace disque.

In [ ]:
# Libérer de l'espace disque avant l'export
import gc
gc.collect()
torch.cuda.empty_cache()

# Nettoyer le cache Colab
!rm -rf /root/.cache/huggingface/hub/*
!rm -rf /content/sample_data

print("✓ Cache nettoyé")
!df -h /content

In [ ]:
# Exporter en GGUF directement sur Google Drive
print(f"📦 Export GGUF vers: {GGUF_DIR}")
print("⏱️ Cette étape peut prendre 10-15 minutes pour Mistral 7B...")

try:
    model.save_pretrained_gguf(
        GGUF_DIR,
        tokenizer,
        quantization_method="q4_k_m",
    )
    print(f"✓ Modèle exporté en GGUF")
    print(f"📁 Fichier sauvegardé sur Drive: {GGUF_DIR}")
except Exception as e:
    print(f"⚠️ Erreur lors de l'export: {e}")
    print("\n💡 Alternative: Sauvegarder les LoRA adapters et merger localement")
    LORA_DIR = f"{SAVE_DIR}/mistral-7b-gw2-lora"
    model.save_pretrained(LORA_DIR)
    tokenizer.save_pretrained(LORA_DIR)
    print(f"✓ LoRA adapters sauvegardés: {LORA_DIR}")
    print("\nPour convertir en GGUF localement:")
    print("1. Télécharger le dossier LoRA depuis Google Drive")
    print("2. pip install llama-cpp-python")
    print("3. python -m llama_cpp.convert --outtype q4_k_m")

In [ ]:
# Lister les fichiers exportés
print("📁 Fichiers sur Google Drive:")
!ls -lh {GGUF_DIR}/ 2>/dev/null || echo "Pas de fichiers GGUF"
print("\n📁 LoRA adapters:")
!ls -lh {SAVE_DIR}/mistral-7b-gw2-lora/ 2>/dev/null || echo "Pas de LoRA adapters"

## 7️⃣ Instructions pour Ollama

### Option A: Avec fichier GGUF
```bash
# 1. Copier le fichier GGUF sur le serveur/PC
scp unsloth.Q4_K_M.gguf user@server:/home/user/models/

# 2. Créer le Modelfile
cat > Modelfile << 'EOF'
FROM /home/user/models/unsloth.Q4_K_M.gguf

TEMPLATE """[INST] {{ .Prompt }} [/INST]
{{ .Response }}</s>"""

PARAMETER temperature 0.2
PARAMETER num_predict 80
PARAMETER num_ctx 1024
PARAMETER stop "</s>"
EOF

# 3. Créer le modèle Ollama
ollama create mistral-gw2 -f Modelfile

# 4. Tester
ollama run mistral-gw2
```

### Dans counter_ai.py:
```python
MODEL_NAME = "mistral-gw2"  # Modèle fine-tuné
```